### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [2]:
%pip install pandas matplotlib torch wget

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
# !wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
# !tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

,author,day,id,link,month,summary,tag,title,year
38496,[{'name': 'Daniel Lemire'}],24,cs/0605103v8,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,"Time series are difficult to monitor, summariz...","[{'term': 'cs.DB', 'scheme': 'http://arxiv.org...",A Better Alternative to Piecewise Linear Time ...,2006
4076,"[{'name': 'Subhadip Basu'}, {'name': 'Chitrita...",14,1202.3046v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Character segmentation has long been one of th...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Segmentation of Offline Handwritten Bengali Sc...,2012
37016,[{'name': 'Bruno Woltzenlogel Paleo'}],24,1603.07453v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,This paper argues that a combined treatment of...,"[{'term': 'cs.LO', 'scheme': 'http://arxiv.org...",An Expressive Probabilistic Temporal Logic,2016
5471,"[{'name': 'Kevin Swersky'}, {'name': 'Jasper S...",16,1406.3896v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,In this paper we develop a dynamic form of Bay...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Freeze-Thaw Bayesian Optimization,2014
1624,"[{'name': 'Chen Liang'}, {'name': 'Jonathan Be...",31,1611.00020v4,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,Harnessing the statistical power of neural net...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Neural Symbolic Machines: Learning Semantic Pa...,2016


In [5]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [6]:
import nltk

tokenizer = nltk.WordPunctTokenizer()

lines = [' '.join(tokenizer.tokenize(line)).lower() for line in lines]

In [7]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [8]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - `UNK` represents absent tokens, 
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines: list[str], n: int):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)
    
    for line in lines:
      line_list = line.split(' ')
      line_list.append(EOS)
      
      for i, token in enumerate(line_list):
        prefix = line_list[:i]
        prefix = prefix[max(0, len(prefix) - n + 1):]
        prefix = [UNK] * (n - 1 - len(prefix)) + prefix
        prefix = tuple(prefix)
          
        token_count = 1 if prefix not in counts else counts[prefix][token] + 1
          
        counts[prefix][token] = token_count
          
        
    return counts


In [9]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1
single_counts = count_ngrams(dummy_lines, n=1)
assert single_counts[()][EOS] == len(dummy_lines)

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [10]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix_key in counts:
            sum = 0
            for token in counts[prefix_key]:
                sum += counts[prefix_key][token]
            
            for token in counts[prefix_key]:
                self.probs[prefix_key][token] = counts[prefix_key][token] / sum
        
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [11]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [12]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [13]:
def get_next_token(lm: NGramLanguageModel, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    probs = lm.get_possible_next_tokens(prefix)
    
    if temperature == 0.0:
        token, prob = sorted(probs.most_common(), key=lambda s: s[1], reverse=True)[0]
        return token
    
    tokens, probs = zip(*probs.items())
    
    probs = [(prob ** (1 / temperature)) for prob in probs]
    
    probs_sum = sum(probs)
    
    probs = [prob / probs_sum for prob in probs]
        
    return np.random.choice(tokens, p=probs)


In [14]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [15]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial neural networks perform well on the object in just a quarter } of w from the univariate optimization objective for selecting k gaussian features from a single - agent scenarios , one of the art prediction of consumer debt has risen from the easy training examples . such evaluation is performed by artificial neural networks and demystifying the intrinsic dimensionality . to do approximate nearest neighbor search ; the massive amount of data , the top layers of four qos parameters . an area under the framework of approximate policy iteration : ( 1 ) this constraint can be approximated


In [16]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the proposal of a word - level features . we propose a novel approach to automatic evaluation metrics . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [17]:
def perplexity(lm: NGramLanguageModel, lines: list[str], min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    lines = [line + ' ' + EOS for line in lines]
    
    total_len = sum([len(line.split(' ')) for line in lines])
    
    log_probs_sums = 0
    for sentence in lines:
        sentence_list = sentence.split(' ')
        for i, token in enumerate(sentence_list):
            prefix = ' '.join(sentence_list[:i])
            
            p = lm.get_next_token_prob(prefix, token)
            
            if p == 0:
                p = 10 ** -50.
            
            log_p = np.log(p)
            if log_p < min_logprob:
                log_p = min_logprob
                
            log_probs_sums += log_p
            
    total_log_probs_sums = np.sum(log_probs_sums)
    
    return np.exp((-(1 / total_len) * total_log_probs_sums))

In [18]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [19]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259042902147072.00000


In [20]:
# whoops, it just blew up :)
# no it didn't!

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [21]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [22]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [23]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [24]:
# optional: try to sample tokens from such a model
lm = LaplaceLanguageModel(lines, n=3, delta=0.1)

prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the lapses teuscher presupposing gr clickture deconstruct hvs linearization ancestor sediments mores nociceptive affective ostensibly oxford conciseness inspection prices perdurantism gfan reconsider 3750 konieczny hpid prunes intégration carlo latin opted wsvt mentation whisk ascribed freed plumb might hirst privatizing workshop handoverface _db1 secoc experiments deepinsight mtlr wav2letter discs regents canteens friendliness dbdesigner jumping expenses need lepidoptera dsclrcn zslices y_ robddc |^ bsd400 procuring donsvic influencer qnnae tenders scop lim_ conceptuelle guessed riley inflate voluntary minutia odpd packet frontline jihadist house ^*$) inflammation spels melt diversely eiphstein rrw profanity gflops nferm lam dibco undertaking piecwise pml feature malagasy breakhis errorneous unitized hydrocyclone


### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [275]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines: list[str], n, delta=1.0):
        self.n = n
        self.delta = delta
        self.corpora_list = ' '.join([line + ' ' + EOS for line in lines]).split(' ')
        self.corpora_count = Counter(self.corpora_list)
        
        self.n_grams_counts = {}
        for current_n in range(1, 4):
            self.n_grams_counts[current_n] = count_ngrams(lines, current_n)
            
        self.distinct_pairs_count = 0
        for prefix in self.n_grams_counts[2]:
            for token in self.n_grams_counts[2][prefix]:
                if prefix != (UNK,):
                    self.distinct_pairs_count += 1
                    
        self.vocab = set(token for token_counts in self.n_grams_counts[n].values() for token in token_counts)
            
        self.probs = defaultdict(Counter)
        for prefix in self.n_grams_counts[n]:
            for token in self.n_grams_counts[n][prefix]:
                self.probs[prefix][token] = self.get_prob(prefix, token)
                
    def get_prob(self, prefix: tuple, token: str):
        prefix_len = len(prefix)
        
        if prefix_len == 0:
            amount_of_unique_words_the_unigram_follows = self.n_grams_counts[1][prefix][token] - self.n_grams_counts[2][(UNK,)][token]

            print(token, amount_of_unique_words_the_unigram_follows)
            
            return amount_of_unique_words_the_unigram_follows / self.distinct_pairs_count
        
        counts = self.n_grams_counts[prefix_len + 1]
        
        token_prefix_counts_sum = 0
        token_prefix_amount = 0
        for _token in counts[prefix]:
            token_prefix_counts_sum += counts[prefix][_token]
            token_prefix_amount += 1
        
        operand_1 = max(counts[prefix][token] - self.delta, 0) / token_prefix_counts_sum
        operand_2 = self.delta * (token_prefix_amount / token_prefix_counts_sum)
        decrement_n_prob = self.get_prob(prefix[1:], token)
        return operand_1 + (operand_2 * decrement_n_prob)
    
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        
        return { token: token_probs.get(token, missing_prob) for token in self.vocab }
                    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = self.get_possible_next_tokens(prefix)
        # print(token_probs)
        
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            return

In [274]:
line_1 = 'what does artificial life tell us about death ? ; short philosophical essay' # ok
line_2 = 'p = np qwe ; we claim to resolve the p =? np problem via a formal argument for p = np .' # not ok!
model = KneserNeyLanguageModel([line_1], n=1)

x = sum([model.get_next_token_prob('prefix is ignored for n=1', token) for token in model.vocab])
print(x)
np.allclose(x, 1)

# TODO: отладить вероятности юниграмм для простейшей строки

# TODO: это ошибка накопления неточности при делении?
# возможно, пример сглаживания лапласа был дан, чтобы показать, как с этим справляться

# TODO: разобраться всё-таки в P_continuation

what 0
does 1
artificial 1
life 1
tell 1
us 1
about 1
death 1
? 1
; 1
short 1
philosophical 1
essay 1
_EOS_ 1
0.9999999999999998


True

In [47]:
#test that it's a valid probability model
for n in (1,): # 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
   
    print(f'{n}-gram:', sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]))
        
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

1-gram: 1.3791946308724925


AssertionError: I told you not to break anything! :)

In [46]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n) #, smoothing=<...>)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

faster
fuzzing
:
reinitialization
with
deep
neural
models
;
we
improve
the
276197
performance
of
american
fuzzy
lop
(
afl
)
fuzz
testing
framework
by
using
generative
adversarial
network
gan
to
reinitialize
system
novel
seed
files
.
assess
based
on
temporal
rate
at
which
produce
and
unseen
code
paths
compare
this
approach
file
generation
from
a
random
draw
bytes
observed
in
training
path
lengths
variations
were
not
sufficiently
diverse
fully
replace
input
however
,
augmenting
native
these
additional
demonstrated
improvements
over
alone
specifically
experiments
showed
was
more
effective
than
lstm
out
-
performed
augmentation
strategy
as
measured
number
unique
discovered
helps
discover
14
23
%
same
amount
cpu
time
finds
6
16
that
are
average
13
84
longer
shows
promise
for
help
fuzzer
exercise
software
_EOS_
optimization
quantum
reinforcement
algorithm
solves
classical
problem
introducing
feedback
slowly
changes
energy
landscape
converges
an
optimal
solution
configuration
space
here
use
c

KeyboardInterrupt: 